# This notebook will serve as the interactive notebook for leveraging the SNAP Graph Processing Tool using the subprocess library. 
- Allows for easier bulk processing of many interferograms, which is needed for non-zero closure phase correction (InSAR triplets)
- Allows for easier bulk processing of custom GRD-like products (will produce both calibrate and uncalibrated backscatter data for classification)

In [1]:
# import dependencies
import os
from time import perf_counter
import subprocess
import shutil

# Functions

In [ ]:
def project_dir(cwd, project_name):
    work_dir = os.path.join(cwd, project_name)
    os.makedirs(work_dir, exist_ok=True)
    data_dir = os.path.join(work_dir,'0_initial')
    os.makedirs(data_dir, exist_ok=True)

    return work_dir, data_dir

def get_triplets(slc_list):
    """
    this program will generate a dictionary with keys triplet_n, where n is the triplet stack
    each triplet_n contains 4 sets. Each set contains (path/to/ref.xml, path/to/sec.xml)
    
    slc_zips = list of directories containing the topsApp, reference, and secondary .xml files    
    created an returned using the project_dir fucntion
    """
    
    triplet_dict={}
    
    for i in range(len(slc_list) - 2):
        triplet_dict[f'triplet_{i+1}'] = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]))
        if i == 0:
            test_triplet = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]), (slc_list[i], slc_list[i+2]))
        else:
            continue
    return triplet_dict, test_triplet

def split_SLC(sourcefile,subswath, polarization, outfile):
    split_cmd = [
        GPT_PATH,
        "TOPSAR-Split", # operation name
        f"-Ssource={sourcefile[6:]}", # SLC file
        f"-Psubswath={subswath}", #1, 2, 3
        f"-PselectedPolarisations={polarization}", # bands being processed
        "-t", outfile, # keep separate, output file location
        "-q", str(MAX_CORES),
        "-c", MAX_MEM
    ]

    # Run the command
    try:
        result = subprocess.run(split_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Command executed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error code {e.returncode}")
        print(f"Error message: {e.stderr}")

def orbit_SLC(sourcefile, orbittype, outfile):
    orbit_cmd = [
        GPT_PATH,
        "Apply-Orbit-File", # operation name
        f"-Ssource={sourcefile}",
        f"-PorbitType={orbittype}",
        "-t", outfile, # keep separate, output file location
        "-q", str(MAX_CORES),
        "-c", MAX_MEM
    ]

    # Run the command
    try:
        result = subprocess.run(orbit_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Command executed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error code {e.returncode}")
        print(f"Error message: {e.stderr}") 

# Steps?

1. split, orbit, coreg, ESD
2. interferogram (topo phase removal, deburst, filter, subset)
4. snaphu export unwrap import
5. geocode, displacement

# Establish needed paths for notebook

In [3]:
#establish working directory, where your SLCs were stored
# will also be where the processing steps for the interferograms and GRD-like products are stored
work_dir, data_dir = project_dir('/mnt/d/SabineRS', 'Sentinel-1')

In [4]:
# get your path to the SNAP GPT install
# typically found in your 'esa-snap' install file in the 'bin' subdirectory
# Linux example: ~/esa-snap/bin/gpt
# WSL example: /mnt/c/Program Files/bin/gpt

# Define paths and parameters
GPT_PATH = '/mnt/c/Program Files/esa-snap/bin/gpt.exe'  # Path to SNAP's gpt executable
INPUT_DIR = data_dir  # Directory containing Sentinel-1 SLCs
OUTPUT_DIR = os.path.join(work_dir, 'test')  # Directory to save interferograms
os.makedirs(OUTPUT_DIR, exist_ok=True)
MAX_CORES = 12  # Adjust based on system capacity
MAX_MEM = '24G'  # Maximum memory for SNAP in bytes
TILE_SIZE = 4096  # Tile size for SNAP

In [5]:
# List all Sentinel-1 SLC files
slc_files = sorted([os.path.join(INPUT_DIR, f) for f in os.listdir(INPUT_DIR) if f.endswith(".zip")])
print("Available Sentinel-1 SLC Files:")
for i, file in enumerate(slc_files):
    print(f"{i + 1}: {file}")

# Select SLC pairs (e.g., manually choose indices for master and slave)
master_index = 0  # Update this to the index of the desired master file
slave_index = 1   # Update this to the index of the desired slave file

master_file = slc_files[master_index]
slave_file = slc_files[slave_index]

print(f"\nSelected Master: {master_file}")
print(f"Selected Slave: {slave_file}")

Available Sentinel-1 SLC Files:
1: /mnt/d/SabineRS/Sentinel-1/0_initial/S1A_IW_SLC__1SDV_20170402T001815_20170402T001842_015958_01A507_82BB.zip
2: /mnt/d/SabineRS/Sentinel-1/0_initial/S1A_IW_SLC__1SDV_20170414T001816_20170414T001843_016133_01AA58_ED18.zip
3: /mnt/d/SabineRS/Sentinel-1/0_initial/S1A_IW_SLC__1SDV_20170508T001817_20170508T001844_016483_01B503_49B5.zip
4: /mnt/d/SabineRS/Sentinel-1/0_initial/S1A_IW_SLC__1SDV_20170520T001818_20170520T001845_016658_01BA5D_60AB.zip
5: /mnt/d/SabineRS/Sentinel-1/0_initial/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.zip
6: /mnt/d/SabineRS/Sentinel-1/0_initial/S1A_IW_SLC__1SDV_20170613T001819_20170613T001846_017008_01C531_BB6E.zip
7: /mnt/d/SabineRS/Sentinel-1/0_initial/S1A_IW_SLC__1SDV_20170625T001820_20170625T001847_017183_01CA88_4D5B.zip
8: /mnt/d/SabineRS/Sentinel-1/0_initial/S1A_IW_SLC__1SDV_20170731T001822_20170731T001849_017708_01DA7E_0990.zip
9: /mnt/d/SabineRS/Sentinel-1/0_initial/S1A_IW_SLC__1SDV_20170905T001823

# Form triplet dictionaries needed for non-zero closure phase InSAR

In [12]:
triplet_dict, example_triplet = get_triplets(sorted([os.path.join(INPUT_DIR, f) for f in os.listdir(INPUT_DIR) if f.endswith(".zip")]))

# 1. TOPSAR-Split

In [20]:
# Example TOPSAR-Split cell, use this to build more cells for the other steps needed
# may make two notebooks; one for interferograms and the other for the custom backscatter products
os.chdir('/mnt/d')
os.makedirs(os.path.join(os.getcwd(), 'SabineRS/Sentinel-1/1_split'), exist_ok=True)

subswath = 'IW3'
polarization = 'VV,VH'

for i, slc in enumerate(example_triplet[0]):
    if i == 0:
        split_file = '/SabineRS/Sentinel-1/1_split/m_testIW3.dim'
    else:
        split_file = '/SabineRS/Sentinel-1/1_split/s_testIW3.dim'
    
    split_SLC(master_file, subswath, polarization, split_file)

Command executed successfully!
Command executed successfully!


# 2. Apply Orbit Files

In [ ]:
os.makedirs(os.path.join(os.getcwd(), 'SabineRS/Sentinel-1/2_orbit'), exist_ok=True)

orbit = 'Sentinel Precise (Auto Download)'

for i , file in enumerate(['/SabineRS/Sentinel-1/1_split/m_testIW3.dim', '/SabineRS/Sentinel-1/1_split/s_testIW3.dim']):
    if i == 0:
        orbit_file = '/SabineRS/Sentinel-1/2_orbit/m_testIW3.dim'
    else:
        orbit_file = '/SabineRS/Sentinel-1/2_orbit/s_testIW3.dim'

    orbit_SLC(split_file, orbit, orbit_file)


shutil.rmtree(os.path.join(os.getcwd(), 'SabineRS/Sentinel-1/1_split'))

Command executed successfully!
Command executed successfully!


# 3. Back-Geocoding

In [ ]:
def geocode_SLC(sourcefile, orbittype, outfile):
    geocode_cmd = [
        GPT_PATH,
        "Apply-Orbit-File", # operation name
        f"-Ssource={sourcefile}",
        f"-PorbitType={orbittype}",
        "-t", outfile, # keep separate, output file location
        "-q", str(MAX_CORES),
        "-c", MAX_MEM
    ]

    # Run the command
    try:
        result = subprocess.run(geocode_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Command executed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error code {e.returncode}")
        print(f"Error message: {e.stderr}") 

In [ ]:
os.makedirs(os.path.join(os.getcwd(), 'SabineRS/Sentinel-1/3_backgeocode'), exist_ok=True)





# 4. ESD

# 5. Interferogram formation

# 6. TOPSAR-Deburst

# 7. Topographic Phase Removal

# 8. Goldstein Phase Filtering

# 9. Subset

# 10. SNAPHU Export